In [15]:
import tweepy as tw
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [16]:
key = 'mKb6uHG96jOKmLn2jyM01V4E5'
secret = 'H83K0AFXVw4HeZAeiobhcIrpCqk4MEozOuCz1zben1BeyKdvao'
access_token = '1505745201943363586-4oVjqFnf27jGUEEf9Zb2IqiyqYBSUy'
access_token_secret = 'ZIJRkUjEf8lNzBHFsxO8baKvFlyI5V0oISPKQSG0L76bY'

In [17]:
# Authentication
auth = tw.OAuthHandler(key,secret)
auth.set_access_token(access_token,access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [18]:
hashtag = '#competition'
query = tw.Cursor(api.search_tweets, q=hashtag).items(1000)
tweets = [{'Tweets':tweet.text, 'Timestamp':tweet.created_at} for tweet in query]
tweets

[{'Tweets': 'RT @heavenskincare: I’m doing a very special #competition to come to #heavenhQ soon look out for the prompt',
  'Timestamp': datetime.datetime(2022, 4, 2, 23, 32, 34, tzinfo=datetime.timezone.utc)},
 {'Tweets': 'RT @MonsterCallsBSC: Monster calls is having a huge give away follow me for your chance to win 500$ winner announced on the 5th April ✅ #cr…',
  'Timestamp': datetime.datetime(2022, 4, 2, 23, 32, 22, tzinfo=datetime.timezone.utc)},
 {'Tweets': 'RT @StuartDorrance: #WIN a copy of my first children’s book Moll - She’s a little scallywag and very mischievous. Your little scallywags wi…',
  'Timestamp': datetime.datetime(2022, 4, 2, 23, 32, 17, tzinfo=datetime.timezone.utc)},
 {'Tweets': 'RT @cosplah_: Check out my OnlyFans!\n👇👇👇👇👇👇👇👇\nhttps://t.co/QAXIMn693N\nI post lewded cosplays!\n—\n#competition #influencer #influencermarketi…',
  'Timestamp': datetime.datetime(2022, 4, 2, 23, 31, 48, tzinfo=datetime.timezone.utc)},
 {'Tweets': "RT @bmstores: 🍃 #COMPETITION TIME 🍃

In [19]:
df = pd.DataFrame.from_dict(tweets)
df.head()

,Tweets,Timestamp
0,RT @heavenskincare: I’m doing a very special #...,2022-04-02 23:32:34+00:00
1,RT @MonsterCallsBSC: Monster calls is having a...,2022-04-02 23:32:22+00:00
2,RT @StuartDorrance: #WIN a copy of my first ch...,2022-04-02 23:32:17+00:00
3,RT @cosplah_: Check out my OnlyFans!\n👇👇👇👇👇👇👇👇...,2022-04-02 23:31:48+00:00
4,RT @bmstores: 🍃 #COMPETITION TIME 🍃\n\nIt's ne...,2022-04-02 23:31:45+00:00


In [20]:
import nltk
from nltk.corpus import stopwords
from textblob import Word, TextBlob

In [21]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = stopwords.words('english')
custom_stopwords = ['RT', '#competition']

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/satviknaik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/satviknaik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/satviknaik/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [22]:
def preprocess_tweets(tweet, custom_stopwords):
    preprocessed_tweet = tweet
    preprocessed_tweet.replace('[^\w\s]','')
    preprocessed_tweet = " ".join(word for word in preprocessed_tweet.split() if word not in stop_words)
    preprocessed_tweet = " ".join(word for word in preprocessed_tweet.split() if word not in custom_stopwords)
    preprocessed_tweet = " ".join(Word(word).lemmatize() for word in preprocessed_tweet.split())
    return(preprocessed_tweet)

df['Preprocessed Tweet'] = df['Tweets'].apply(lambda x:preprocess_tweets(x,custom_stopwords))
df.head()

,Tweets,Timestamp,Preprocessed Tweet
0,RT @heavenskincare: I’m doing a very special #...,2022-04-02 23:32:34+00:00,@heavenskincare: I’m special come #heavenhQ so...
1,RT @MonsterCallsBSC: Monster calls is having a...,2022-04-02 23:32:22+00:00,@MonsterCallsBSC: Monster call huge give away ...
2,RT @StuartDorrance: #WIN a copy of my first ch...,2022-04-02 23:32:17+00:00,@StuartDorrance: #WIN copy first children’s bo...
3,RT @cosplah_: Check out my OnlyFans!\n👇👇👇👇👇👇👇👇...,2022-04-02 23:31:48+00:00,@cosplah_: Check OnlyFans! 👇👇👇👇👇👇👇👇 https://t....
4,RT @bmstores: 🍃 #COMPETITION TIME 🍃\n\nIt's ne...,2022-04-02 23:31:45+00:00,@bmstores: 🍃 #COMPETITION TIME 🍃 It's nearly t...


In [23]:
df['polarity'] = df['Preprocessed Tweet'].apply(lambda x:TextBlob(x).sentiment[0])
df['subjectivity'] = df['Preprocessed Tweet'].apply(lambda x:TextBlob(x).sentiment[1])
df

,Tweets,Timestamp,Preprocessed Tweet,polarity,subjectivity
0,RT @heavenskincare: I’m doing a very special #...,2022-04-02 23:32:34+00:00,@heavenskincare: I’m special come #heavenhQ so...,0.357143,0.571429
1,RT @MonsterCallsBSC: Monster calls is having a...,2022-04-02 23:32:22+00:00,@MonsterCallsBSC: Monster call huge give away ...,0.600000,0.650000
2,RT @StuartDorrance: #WIN a copy of my first ch...,2022-04-02 23:32:17+00:00,@StuartDorrance: #WIN copy first children’s bo...,0.168750,0.433333
3,RT @cosplah_: Check out my OnlyFans!\n👇👇👇👇👇👇👇👇...,2022-04-02 23:31:48+00:00,@cosplah_: Check OnlyFans! 👇👇👇👇👇👇👇👇 https://t....,0.000000,0.000000
4,RT @bmstores: 🍃 #COMPETITION TIME 🍃\n\nIt's ne...,2022-04-02 23:31:45+00:00,@bmstores: 🍃 #COMPETITION TIME 🍃 It's nearly t...,0.100000,0.400000
...,...,...,...,...,...
995,#Best of the day on https://t.co/yDTLVmXMbb\nT...,2022-04-02 20:50:33+00:00,#Best day https://t.co/yDTLVmXMbb This Entrepr...,1.000000,0.650000
996,What's #Best on https://t.co/wNcTtVfEg3 ?\nThi...,2022-04-02 20:50:27+00:00,What's #Best https://t.co/wNcTtVfEg3 ? This En...,1.000000,0.650000
997,RT @cherrylaneGC: 😍 GIVEAWAY! 😍\n\nWho's ready...,2022-04-02 20:50:03+00:00,@cherrylaneGC: 😍 GIVEAWAY! 😍 Who's ready easte...,0.370000,0.480000
998,RT @PlaytexUK: Win! We have two sets of our go...,2022-04-02 20:49:45+00:00,@PlaytexUK: Win! We two set gorgeous Essential...,0.425000,0.400000
